In [1]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

In [2]:
vest_mo_18 = gp.read_file("./raw-from-source/VEST/mo_2018/mo_2018.shp")

In [3]:
print(vest_mo_18.head())

  STATEFP COUNTYFP           NAME  G18USSRHAW  G18USSDMCC  G18USSLCAM  \
0      29      189          MR032          77          26           0   
1      29      189  CHE044/LAF001         254         288           2   
2      29      201       Commerce         215          89           1   
3      29      043    West Finley        1254         504          34   
4      29      189  MID036/MID048          48         257           1   

   G18USSGCRA  G18USSIODE  G18AUDRMCD  G18AUDDGAL  G18AUDLOTO  G18AUDGFIT  \
0           0           0          57          42           0           0   
1           2           2         227         303           7           0   
2           0           0         197          89           5           0   
3          14          38        1056         651          67          15   
4           4           9          37         268           5           4   

   G18AUDCLUE                                           geometry  
0           2  POLYGON ((-90.54

## Election Data Sources

Election results from the Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-mo/tree/master/2018). Absentee, provisional, and mail ballots were reported countywide in nearly every county; these were distributed by candidate to precincts based on their share of the precinct-level reported vote.

- G18USSRHAW - Josh Hawley (Republican Party)  
- G18USSDMCC - Claire McCaskill (Democratic Party)  
- G18USSLCAM - Japheth Campbell (Libertarian Party)  
- G18USSGCRA - Jo Crain (Green Party)  
- G18USSIODE - Craig O'Dear (Independent)  
  
  
- G18AUDRMCD - Saundra McDowell (Republican Party)  
- G18AUDDGAL - Nicole Galloway (Democratic Party)  
- G18AUDLOTO - Sean O'Toole (Libertarian Party)  
- G18AUDGFIT - Don Fitz (Green Party)  
- G18AUDCLUE - Jacob Luetkemeyer (Constitution Party)  


In [4]:
mo_elections = pd.read_csv("./raw-from-source/Open_Elections/20181106__mo__general__precinct.csv")
candidate_list = ['Josh Hawley','Claire McCaskill','Japheth Campbell','Jo Crain',
 "Craig O'Dear",'Saundra McDowell','Nicole Galloway',"Sean O'Toole",'Don Fitz','Jacob Luetkemeyer']
office_list = ['U.S. Senate', 'State Auditor']
mo_elections["candidate"] = mo_elections["candidate"].str.strip()
mo_elections = mo_elections[mo_elections["office"].isin(office_list)]
mo_elections = mo_elections[mo_elections["candidate"].isin(candidate_list)]


In [5]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Missouri"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
#print(fips_file[~fips_file["County Name"].isin(mo_elections["county"])]["County Name"].unique())
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
mo_changes = {'De Kalb':'Dekalb','Kansas City':'Jackson','McDonald':'Mcdonald',
              'St. Charles':'St Charles','St. Clair':'St Clair',
              'Ste. Genevieve':'Ste Genevieve','St. Francois':'St Francois',
              'St. Louis City':'St Louis City','St. Louis County':'St Louis'}
mo_elections['county'] = mo_elections['county'].map(mo_changes).fillna(mo_elections['county'])
mo_elections['COUNTYFP'] = mo_elections['county'].map(fips_dict).fillna(mo_elections['county'])
mo_elections["unique_ID"]=mo_elections["COUNTYFP"]+mo_elections["precinct"]

In [6]:
mo_elections_pivot = pd.pivot_table(mo_elections,index=["unique_ID","COUNTYFP","precinct"],columns=["candidate"],values=['votes'],aggfunc=sum)
mo_elections_pivot = mo_elections_pivot.fillna(0)
mo_elections_pivot.columns = mo_elections_pivot.columns.droplevel(0)
mo_elections_pivot.reset_index(drop=False,inplace=True)
mo_elections_pivot = mo_elections_pivot.rename(columns={'Claire McCaskill':"G18USSDMCC","Craig O'Dear": "G18USSIODE",'Don Fitz':"G18AUDGFIT",'Jacob Luetkemeyer':"G18AUDCLUE",'Japheth Campbell':"G18USSLCAM",'Jo Crain':"G18USSGCRA",'Josh Hawley':"G18USSRHAW",'Nicole Galloway':"G18AUDDGAL",'Saundra McDowell':"G18AUDRMCD","Sean O'Toole":"G18AUDLOTO"})

In [7]:
matcher_increase = pd.read_csv("./matcher_increase.csv")
new_name_dict = dict(zip(matcher_increase['source_names'], matcher_increase['vest_names']))
mo_elections_pivot["unique_ID"] = mo_elections_pivot["unique_ID"].str.strip()
mo_elections_pivot["unique_ID"] = mo_elections_pivot["unique_ID"].map(new_name_dict).fillna(mo_elections_pivot["unique_ID"])
kc_precincts = mo_elections_pivot[mo_elections_pivot["unique_ID"].str[0:5]=="095KC"]

In [8]:
print(kc_precincts.columns)

Index(['unique_ID', 'COUNTYFP', 'precinct', 'G18USSDMCC', 'G18USSIODE',
       'G18AUDGFIT', 'G18AUDCLUE', 'G18USSLCAM', 'G18USSGCRA', 'G18USSRHAW',
       'G18AUDDGAL', 'G18AUDRMCD', 'G18AUDLOTO'],
      dtype='object', name='candidate')


In [9]:
remove_list = ["ABSENTEE","PROVISIONAL","FEDERAL","FEDERAL/STATE","INTRASTATE","WRITE-IN","CENTRAL POLLING LOCATION","CENTRAL POLL","ABSENTEE1","ABSENTEE2","VOTE CENTER","CENTRAL POLLING LOCATION: MSU","INTRASTATE/NEW RESIDENT","WRITE-INS","MULTI-PRECINCT COLLECTION","INTRASTATE NEW RESIDENT","MULTI-PRECINCT COLLECTION","ABSENTEE, PROVISIONAL & CENTRAL","INTRASTATE01","INTRASTATE02"]
mo_elections_countywide = mo_elections_pivot[(mo_elections_pivot["precinct"].isin(remove_list)) | (mo_elections_pivot["unique_ID"]=='019CENTRAL')]
mo_elections_pivot = mo_elections_pivot[~(mo_elections_pivot["precinct"].isin(remove_list)) & ~(mo_elections_pivot["unique_ID"]=='019CENTRAL') & ~(mo_elections_pivot["unique_ID"].str[0:5]=="095KC")]
print(mo_elections_pivot[(mo_elections_pivot["unique_ID"].str[0:5]=="095KC")])
countywide_totals = mo_elections_countywide.groupby(["COUNTYFP"]).sum()
precinct_totals = mo_elections_pivot.groupby(["COUNTYFP"]).sum()
data_to_add = pd.merge(precinct_totals,countywide_totals,on=['COUNTYFP'],how='left',indicator=False)
data_to_add.reset_index(inplace=True,drop=False)

Empty DataFrame
Columns: [unique_ID, COUNTYFP, precinct, G18USSDMCC, G18USSIODE, G18AUDGFIT, G18AUDCLUE, G18USSLCAM, G18USSGCRA, G18USSRHAW, G18AUDDGAL, G18AUDRMCD, G18AUDLOTO]
Index: []


In [10]:

joined = pd.merge(mo_elections_pivot,data_to_add,on=['COUNTYFP'],how='left',indicator=False)
joined = joined.fillna(0)
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    precinct_totals = i + "_x"
    countywide_reported = i+"_y"
    joined[i]=round(joined[i]+((joined[i]*joined[countywide_reported])/joined[precinct_totals]),0)
joined = joined [["unique_ID","COUNTYFP","precinct","G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]]
joined = joined.append(kc_precincts)
print(joined[joined["unique_ID"].str[0:5]=="095KC"])

candidate          unique_ID COUNTYFP       precinct  G18USSDMCC  G18USSIODE  \
1076         095KC 1910/1911      095   W-19-P11, 10       380.0         6.0   
1077           095KC 101/104      095       W1-P1, 4      1667.0        28.0   
1078           095KC 102/107      095       W1-P2, 7      1127.0         8.0   
1079               095KC 103      095          W1-P3       658.0        10.0   
1080               095KC 105      095          W1-P5       825.0         7.0   
...                      ...      ...            ...         ...         ...   
1224           095KC 910/911      095     W9-P10, 11      1141.0        13.0   
1225           095KC 901/902      095       W9-P2, 1      1300.0        16.0   
1226       095KC 903/904/906      095    W9-P4, 3, 6      1182.0        13.0   
1227       095KC 905/912/913      095  W9-P5, 12, 13       698.0         8.0   
1228       095KC 907/908/909      095    W9-P8, 7, 9      1168.0         7.0   

candidate  G18AUDGFIT  G18AUDCLUE  G18U

- G18USSRHAW - Josh Hawley (Republican Party)  
- G18USSDMCC - Claire McCaskill (Democratic Party)  
- G18USSLCAM - Japheth Campbell (Libertarian Party)  
- G18USSGCRA - Jo Crain (Green Party)  
- G18USSIODE - Craig O'Dear (Independent)  
  
  
- G18AUDRMCD - Saundra McDowell (Republican Party)  
- G18AUDDGAL - Nicole Galloway (Democratic Party)  
- G18AUDLOTO - Sean O'Toole (Libertarian Party)  
- G18AUDGFIT - Don Fitz (Green Party)  
- G18AUDCLUE - Jacob Luetkemeyer (Constitution Party)  

In [11]:
vest_mo_18["unique_ID"]=vest_mo_18["COUNTYFP"]+vest_mo_18["NAME"]
vest_empty = vest_mo_18[(vest_mo_18['G18USSRHAW']==0) & (vest_mo_18['G18USSDMCC']==0) &
              (vest_mo_18['G18USSLCAM']==0) & (vest_mo_18['G18USSGCRA']==0) &
              (vest_mo_18['G18USSIODE']==0) & (vest_mo_18['G18AUDRMCD']==0) &
              (vest_mo_18['G18AUDDGAL']==0) & (vest_mo_18['G18AUDLOTO']==0) &
              (vest_mo_18['G18AUDGFIT']==0) & (vest_mo_18['G18AUDCLUE']==0)]["unique_ID"]

joined_empty = joined[(joined['G18USSRHAW']==0) & (joined['G18USSDMCC']==0) &
              (joined['G18USSLCAM']==0) & (joined['G18USSGCRA']==0) &
              (joined['G18USSIODE']==0) & (joined['G18AUDRMCD']==0) &
              (joined['G18AUDDGAL']==0) & (joined['G18AUDLOTO']==0) &
              (joined['G18AUDGFIT']==0) & (joined['G18AUDCLUE']==0)]["unique_ID"]

print(joined_empty.head())
print(vest_empty[vest_empty=="189MER44"])

2161    189MER44
Name: unique_ID, dtype: object
Series([], Name: unique_ID, dtype: object)


In [12]:
new_precincts = pd.DataFrame(columns=joined.columns)
new_precincts["unique_ID"]=vest_empty
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    new_precincts[i]=0
joined = joined.append(new_precincts)

In [13]:
print(vest_mo_18.shape)
print(joined.shape)

final_join = pd.merge(vest_mo_18,joined,how="outer",on="unique_ID",indicator=True)
print(final_join["_merge"].value_counts())

print(final_join[final_join["_merge"]=="right_only"].head())
print(final_join[final_join["_merge"]=="left_only"].head())




print(final_join[final_join["_merge"]=="left_only"]["unique_ID"])

(3242, 15)
(3243, 13)
both          3242
right_only       1
left_only        0
Name: _merge, dtype: int64
     STATEFP COUNTYFP_x NAME  G18USSRHAW_x  G18USSDMCC_x  G18USSLCAM_x  \
3242     NaN        NaN  NaN           NaN           NaN           NaN   

      G18USSGCRA_x  G18USSIODE_x  G18AUDRMCD_x  G18AUDDGAL_x  ...  \
3242           NaN           NaN           NaN           NaN  ...   

      G18USSIODE_y  G18AUDGFIT_y  G18AUDCLUE_y G18USSLCAM_y G18USSGCRA_y  \
3242           0.0           0.0           0.0          0.0          0.0   

     G18USSRHAW_y G18AUDDGAL_y  G18AUDRMCD_y  G18AUDLOTO_y      _merge  
3242          0.0          0.0           0.0           0.0  right_only  

[1 rows x 28 columns]
Empty GeoDataFrame
Columns: [STATEFP, COUNTYFP_x, NAME, G18USSRHAW_x, G18USSDMCC_x, G18USSLCAM_x, G18USSGCRA_x, G18USSIODE_x, G18AUDRMCD_x, G18AUDDGAL_x, G18AUDLOTO_x, G18AUDGFIT_x, G18AUDCLUE_x, geometry, unique_ID, COUNTYFP_y, precinct, G18USSDMCC_y, G18USSIODE_y, G18AUDGFIT_y, G18

In [14]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff != 0):
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]['unique_ID'])
                if (diff>max_diff):
                    max_diff = diff
                    print("New max diff is: ", str(max_diff))
                    print(df.iloc[j]['unique_ID'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]['unique_ID'])
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 5")
    diff_list.sort()
    print(diff_list)

In [15]:
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
validater_row(final_join[final_join["_merge"]=="both"],col_list)


New max diff is:  1.0
043West Finley
New max diff is:  2.0
0194K/4M/37
There are  3242  total rows
1067  of these rows have election result differences
2175  of these rows are the same

The max difference between any one shared column in a row is:  2.0
The average difference is:  1.0192307692307692
There are  0 precinct results with a difference greater than 5
['001Brashear', '001Northeast 5', '001Novinger', '001Rural Benton', '001Southeast 2', '001Southwest 1', '003John Glenn/Country Club/Beeler', '003Lincoln', '003Rochester', '005Rock Port', '005Tarkio', '007Community', '007Martinsburg', '007Mexico No. 1', '007Mexico No. 2/3/4', '007Mexico No. 5', '007Mexico No. 6', '007Rush Hill/Benton City', '007South Fork', '009Cassville City', '009Cassville Rural/Mineral', '009Exeter', '009McDowell/Purdy', '009Ozark/Crane Creek', '009Roaring River', '009Shell Knob', '011City 3&4', '011DoylesPort/Milford/Newport/Union', '011Golden City', '011Lamar/Richland', '013Adrian North', '013Adrian South', '

In [16]:

def validater_row_vect(df, county_column, join_column, column_List):
    matching_rows = 0
    different_rows = 0
    
    county_join_cols = (df[join_column].astype('str')).to_numpy()
    
    # it's because before, we were checking that a row was entirely consistent
    # here, we are double checking a lot
    # so, let's just keep one track of the rows that are messed up
    
    wrong_idxs = np.zeros(len(df))
    for i in column_List:
        left_data = df[i + "_x"].to_numpy()
        right_data = df[i + "_y"].to_numpy()
        
        local_idxs = np.where(left_data != right_data)
        wrong_idxs[local_idxs] = 1
        #print("Wrong idxs", wrong_idxs)
        
    # we are close, we get the same result, but are double adding lots of rows
    different_rows += np.sum(wrong_idxs)
    matching_rows += len(df) - different_rows
    
    diff_list = county_join_cols[np.where(wrong_idxs == 1)]
            
    print("There are ", len(df.index)," total rows")
    print(f"{int(different_rows)} of these rows have election result differences")
    print(f"{int(matching_rows)} of these rows are the same")
    print(diff_list)
    print("")

    

col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
validater_row_vect(final_join[final_join["_merge"]=="both"],"unique_ID","unique_ID",col_list)

There are  3242  total rows
1067 of these rows have election result differences
2175 of these rows are the same
['043West Finley' '165Barry East 34' '123Marquand' ...
 '175Jacksonville/Levicks Mill/Cairo' '147Green' '147Nodaway']



## Precinct Shapefile Sources

The documentation mentions 4 different sources for shapefiles:

1. Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release.  
2. Monroe County comes from the 2010 Census VTD release.  
3. Platte County comes from the 2020 Census VTD release.  
4. The following counties instead used maps or shapefiles sourced from the respective county governments: Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.  

### 1. U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release
> Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release.

https://www.census.gov/geo/partnerships/pvs/partnership19v2/st29_mo.html

In [287]:
#When downloading from the Census redistricing data program, these use a FIPS code to identify counties

fips_file["unique_ID"] =  "29" + fips_file["FIPS County"]
fips_codes = fips_file["unique_ID"].tolist()

#Combine all the data from separate files into one
li = []
for i in fips_codes:
    ref = "./raw-from-source/Census/2020_Phase_2/partnership_shapefiles_19v2_"
    file_ref = ref+i+"/PVS_19_v2_vtd_"+i+".shp"
    file_prev = gp.read_file(file_ref)
    #print(file_prev.shape)
    li.append(file_prev)
global shapefiles_census
shapefiles_census = pd.concat(li, axis=0, ignore_index=True)
print(shapefiles_census.shape)

(4607, 16)


In [288]:
#print(shapefiles_census.head())
county_list = ["Audrain", "Bates","Caldwell","Callaway","Camden","Cape Girardeau","Cedar","Cooper","Franklin","Greene","Jackson","Jasper","Jefferson", 
               "Laclede", "Lafayette","Lawrence", "Marion", "Mcdonald","Miller","Monroe","Nodaway",
               "Osage", "Ozark", "Pemiscot","Platte", "Pike", "Randolph", "Scott","Ste Genevieve", "Texas", "Warren", "Washington", "Worth", "Wright"]
print(len(county_list))
fips_to_take = fips_file[fips_file["County Name"].isin(county_list)]
different_source = fips_to_take["FIPS County"].tolist()
print(different_source)

shapefiles_census = shapefiles_census[~shapefiles_census["COUNTYFP"].isin(different_source)]

34
['007', '013', '025', '027', '029', '031', '039', '053', '071', '077', '095', '097', '099', '105', '107', '109', '119', '127', '131', '137', '147', '151', '153', '155', '163', '165', '175', '186', '201', '215', '219', '221', '227', '229']


In [289]:
#Filter down election results to just these counties
print(joined.shape)
elections_census_shapes = joined[~joined["COUNTYFP"].isin(different_source)]
print(elections_census_shapes.shape)

#Create a unique ID
shapefiles_census["unique_ID"] = shapefiles_census["COUNTYFP"]+shapefiles_census["NAME"]
shapefiles_census["unique_ID"] = shapefiles_census["unique_ID"].apply(lambda x: x.title() if (x[0:3]!="189" and x[0:3]!="510" and x[0:3]!="047" and x[0:3]!="159" and x[0:3]!="051" and x[0:3]!="217" and x[0:3]!="183" ) else x)



(3243, 13)
(2259, 13)


In [290]:
census_name_changes_dict = {"003John Glenn A":"003John Glenn","009Capps Creek":"009Monett/Capps Creek",
 "017Drum-Dongola":"017Drum Dungola",
 "021Bible Baptist Temple":"021Riverside Baptist Church",
'037No. 1 - Sherman':'03701 Sherman',
'037No. 7 - Coldwater':'03707 Drexel',
'037No. 10 - Grand River':'03710 Grand River Rural',
'037No. 11 - Grand River, Ward 1':'03711 Harrisonville Ward 1',
'037No. 12 - Grand River, Ward 2':'03712 Harrisonville Ward 2',
'037No. 13 - Grand River, Ward 3':'03713 Harrisonville Ward 3',
'037No. 14 - Grand River, Ward 4':'03714 Harrisonville Ward 4',
'037No. 19 - Polk':'03719 Polk',
'037No. 20 - Pleasant Hill':'03720 Pleasant Hill',
'037No. 21 - Pleasant Hill':'03721 Pleasant Hill East',
'037No. 22 - Pleasant Hill':'03722 Pleasant Hill',
'037No. 23 - Pleasant Hill':'03723 Pleasant Hill Rural',
'037No. 24 - Peculiar':'03724 Peculiar Rural',
'037No. 25 - West Peculiar':'03725 West Peculiar Rural',
'037No. 26 - West Peculiar':'03726 West Peculiar City',
'037No. 27 - Union':'03727 Union',
'037No. 28 - Big Creek':'03728 Big Creek Rural',
'037No. 29 - Big Creek':'03729 Winnebago',
'037No. 30 - Raymore':'03730 Raymore Rural',
'037No. 31 - Raymore, Ward 3':'03731 Raymore Ward 3',
'037No. 32 - Raymore, Ward 2':'03732 Raymore Ward 2',
'037No. 33 - Raymore, Ward 2':'03733 Raymore Ward 2',
'037No. 34 - Raymore, Wards 3 & 4':'03734 Raymore Wards 3 & 4',
'037No. 35 - Raymore, Ward 4':'03735 Raymore Ward 4',
'037No. 36 - Kansas City':'03736 Mt Pleasant Rural',
'037No. 37 - Mt Pleasant':'03737 Pleasant Prairie',
'037No. 38 - Mt Pleasant, NE Ward 1':'03738 Belton NE Ward 1',
'037No. 39 - Mt Pleasant, NE Ward 1':'03739 Belton NE Ward 1',
'037No. 40 - Mt Pleasant, NW Ward 2':'03740 Belton NW Ward 2',
'037No. 41 - Mt Pleasant, NW Ward 2':'03741 Belton NW Ward 2',
'037No. 42 - Mt Pleasant, SW Ward 3':'03742 Belton SW Ward 3',
'037No. 43 - Mt Pleasant, SW Ward 3':'03743 Belton SW Ward 3',
'037No. 44 - Mt Pleasant, SW Ward 4':'03744 Belton SW Ward 4',
'037No. 45 - Mt Pleasant, SE Ward 4':'03745 Belton SE Ward 4',
'037No. 46 - Mt Pleasant':'03746 East Creek',
'037No. 47 - Raymore, Ward 1':'03747 Raymore Ward 1',
'145Neosho No. 3':'145Neosho #3',
 "043Cassidy":"043Cassidy 1","510ST 1-4":"510STL 1-4",
'181Flatwoods & Johnson P2 & Washington':'181Flatwoods/Johnson P2/Washington',
'181Gatewood & Union':'181Gatewood/Union',
'181Harris & Varner P1':'181Harris/Varner P1',
'181Shirley & Kelly & Pine':'181Shirley/Kelly/Pine',
'181Thomas & Varner P2':'181Thomas/Varner P2'}

shapefiles_census["unique_ID"] = shapefiles_census["unique_ID"].map(census_name_changes_dict).fillna(shapefiles_census["unique_ID"])



In [291]:
shapefiles_census_crs = shapefiles_census.crs
print(shapefiles_census.columns)
shapefiles_census = shapefiles_census[['geometry', 'unique_ID']]
print(shapefiles_census.head())

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAMELSAD', 'VTDI', 'LSAD', 'CHNG_TYPE',
       'ORIG_NAME', 'ORIG_CODE', 'RELATE', 'NAME', 'VINTAGE', 'FUNCSTAT',
       'JUSTIFY', 'MTFCC', 'geometry', 'unique_ID'],
      dtype='object')
                                            geometry       unique_ID
0  POLYGON Z ((-92.51637 40.23389 0.00000, -92.51...     001Brashear
1  POLYGON Z ((-92.58690 40.19287 0.00000, -92.58...  001Northeast 4
2  POLYGON Z ((-92.58702 40.20330 0.00000, -92.58...  001Northeast 5
3  POLYGON Z ((-92.58912 40.24646 0.00000, -92.58...  001Northeast 6
4  POLYGON Z ((-92.85637 40.06093 0.00000, -92.85...     001Novinger


In [292]:
#Counties to deal with manually

#003
#009
#015
#017
#019
#035
#043
#143
#189

#Can't deal with
#011

dict_func_calls = {'03716 East Lynne/18 Gunn City':["037No. 16 - Camp Branch","037No. 18 - Index"],
'03702 Dayton/03 Garden City/15 Camp Branch/17 Index':['037No. 2 - Dayton','037No. 3 - Garden City','037No. 15 - Camp Branch','037No. 17 - Index'],
'03704 Austin/05 Archie/06 Everett':["037No. 4 - Austin","037No. 5 - Austin","037No. 6 - Everett"],
'03708 West Dolan/09 Dolan':["037No. 8 - West Dolan","037No. 9 - Dolan"],
'1591st Ward':['1591st Ward 1st Precinct','1591st Ward 2nd Precinct','1591st Ward 3rd Precinct'],
'1592nd Ward':['1592nd Ward 3rd Precinct','1592nd Ward 1st Precinct','1592nd Ward 2nd Precinct'],
'1593rd Ward':['1593rd Ward 1st Precinct','1593rd Ward 2nd Precinct','1593rd Ward 3rd Precinct'],
'1594th Ward':['1594th Ward 1st Precinct','1594th Ward 2nd Precinct','1594th Ward 3rd Precinct'],
'159Smithton W BP/Smithton E & W/Lake Creek':['159Smithton E & W/Lake Creek','159Smithton W BP' ],
'181Poynor/Current River/West Doniphan/Doniphan P1/Doniphan P2':['181Doniphan P1 & Doniphan P2','181Poynor & Current River & West Doniphan'],
'185Appleton':['185Appleton No. 1','185Appleton No. 2'],
'185Butler':['185Butler East','185Butler West'],
'185Collins/Washington':['185Collins','185Washington'],
'185Dallas/Doyal':['185Dallas','185Doyal'],
'185Monegaw/Osage':['185Monegaw','185Osage'],
'185Osceola/Polk':['185Osceola East','185Osceola West','185Polk'],
'185Center/Roscoe':[ '185Center','185Roscoe'],
'185Speedwell/Taber':['185Speedwell','185Taber'],
'187Farmington No. 4/Busiek':['187Busiek','187Farmington No. 4'],
'197Coffey/Downing':['197Coffey','197Downing'],
'197North Liberty/Coatsville/Glenwood/South Liberty':['197Coatsville','197Glenwood','197North Liberty','197South Liberty'],
'197East Prairie/West Prairie':['197East Prairie','197West Prairie'],
'203Spring Creek/Birch Tree/Bartlett':['203Spring Creek','203Bartlett','203Birch Tree'],
'203Eminence/Delaware':['203Delaware','203Eminence'],
'205BETHEL/TIGER FORK':['205Bethel/Tiger Fork',],
'207Bloomfield Ward No. 1/2':['207Bloomfield Ward No. 2','207Bloomfield Ward No. 1'],
'211Green City/Pennville':['211Green City','211Pennville'],
'211Taylor/Humphreys':['211Taylor','211Humphreys'],
'211Pollock/Boynton':['211Boynton','211Pollock'],
'211South Polk/Reger/Pleasant Hill':['211Reger','211South Polk','211Pleasant Hill'],
'211West Polk/Asbury':['211West Polk','211Asbury'],
"223Peach Tree/Piedmont Ward 1/Ward 2":['223Peach Tree','223Piedmont Ward No. 1','223Piedmont Ward No. 2'],
"223Wappapello/McGee/Shook":['223Wappapello','223McGee','223Shook'],
"223Coldwater/Greenville/Silva":['223Coldwater','223Greenville','223Silva'],
"223Cool Springs/Chaonia":['223Chaonia','223Cool Springs'],
"223Lowndes/Hiram":['223Lowndes','223Hiram'],
"223Patterson/Brunot/Smith Mill":['223Patterson','223Smith Mill','223Brunot'],
"223Cascade/Clubb":['223Clubb','223Cascade'],
"225East Ozark/Northview A":["225Northview A","225East Ozark"],
"225Finley/Hazelwood":["225Finley","225Hazelwood"],
"225Niangua/Union":["225Niangua","225Union"],
"225Washington/Grant":["225Washington","225Grant"]}

In [293]:
def precinct_merger(precinct_list,new_precinct):
    global shapefiles_census
    prec_holder = []
    for i in precinct_list:
        prec_holder.append(shapefiles_census.loc[shapefiles_census["unique_ID"]==i])
    #Create a merged geometry value
    new_geom = pd.concat(prec_holder, ignore_index=True).unary_union
    #print(new_geom)
    new_prec = gp.GeoDataFrame(geometry=[new_geom],columns=["geometry","unique_ID"],crs = shapefiles_census_crs)
    new_prec["unique_ID"]=new_precinct
    shapefiles_census = shapefiles_census.append(new_prec)
    #Delete the old precincts
    print(shapefiles_census.tail())
    shapefiles_census = shapefiles_census[~shapefiles_census["unique_ID"].isin(precinct_list)]

In [294]:
for val in dict_func_calls:
    precinct_merger(dict_func_calls[val],val)
    


                                               geometry  \
4603  POLYGON Z ((-90.27595 38.62261 0.00000, -90.27...   
4604  POLYGON Z ((-90.26179 38.59017 0.00000, -90.26...   
4605  POLYGON Z ((-90.26480 38.64410 0.00000, -90.26...   
4606  MULTIPOLYGON Z (((-90.20242 38.65626 0.00000, ...   
0     POLYGON Z ((-94.18267 38.70109 0.00000, -94.18...   

                          unique_ID  
4603                   510STL 10-13  
4604                   510STL 14-10  
4605                   510STL 28-11  
4606                     510STL 3-9  
0     03716 East Lynne/18 Gunn City  
                                               geometry  \
4604  POLYGON Z ((-90.26179 38.59017 0.00000, -90.26...   
4605  POLYGON Z ((-90.26480 38.64410 0.00000, -90.26...   
4606  MULTIPOLYGON Z (((-90.20242 38.65626 0.00000, ...   
0     POLYGON Z ((-94.18267 38.70109 0.00000, -94.18...   
0     POLYGON Z ((-94.25258 38.47071 0.00000, -94.25...   

                                              unique_ID  
4604

                                            geometry              unique_ID
0  POLYGON Z ((-93.72050 38.20528 0.00000, -93.71...              185Butler
0  POLYGON Z ((-93.67942 37.91783 0.00000, -93.67...  185Collins/Washington
0  POLYGON Z ((-93.61960 38.00346 0.00000, -93.61...        185Dallas/Doyal
0  POLYGON Z ((-93.94946 38.00406 0.00000, -93.94...       185Monegaw/Osage
0  POLYGON Z ((-93.70332 38.09281 0.00000, -93.69...        185Osceola/Polk
                                            geometry              unique_ID
0  POLYGON Z ((-93.67942 37.91783 0.00000, -93.67...  185Collins/Washington
0  POLYGON Z ((-93.61960 38.00346 0.00000, -93.61...        185Dallas/Doyal
0  POLYGON Z ((-93.94946 38.00406 0.00000, -93.94...       185Monegaw/Osage
0  POLYGON Z ((-93.70332 38.09281 0.00000, -93.69...        185Osceola/Polk
0  POLYGON Z ((-93.89996 37.90026 0.00000, -93.89...       185Center/Roscoe
                                            geometry           unique_ID
0  POLYGON Z ((

                                            geometry  \
0  POLYGON Z ((-93.02571 40.15285 0.00000, -93.02...   
0  POLYGON Z ((-93.19701 40.16497 0.00000, -93.19...   
0  POLYGON Z ((-90.77923 37.12721 0.00000, -90.77...   
0  POLYGON Z ((-90.35157 37.00005 0.00000, -90.35...   
0  POLYGON Z ((-90.49986 37.05634 0.00000, -90.49...   

                              unique_ID  
0     211South Polk/Reger/Pleasant Hill  
0                   211West Polk/Asbury  
0  223Peach Tree/Piedmont Ward 1/Ward 2  
0             223Wappapello/McGee/Shook  
0         223Coldwater/Greenville/Silva  
                                            geometry  \
0  POLYGON Z ((-93.19701 40.16497 0.00000, -93.19...   
0  POLYGON Z ((-90.77923 37.12721 0.00000, -90.77...   
0  POLYGON Z ((-90.35157 37.00005 0.00000, -90.35...   
0  POLYGON Z ((-90.49986 37.05634 0.00000, -90.49...   
0  POLYGON Z ((-90.49839 36.93901 0.00000, -90.49...   

                              unique_ID  
0                   211West Polk

In [310]:
county_fips_code_list = ["003","009","015","017","019","035","043","143","189"]
filtered_census = elections_census_shapes[elections_census_shapes["unique_ID"].str[0:3].isin(county_fips_code_list)]["unique_ID"]
for val in filtered_census:
    if "/" in val:
        pieces = val.split("/")
        fips = pieces[0][0:3]
        holder = []
        holder.append(pieces[0])
        for i in range(1,len(pieces)):
            holder.append(fips+pieces[i])
        precinct_merger(holder,val)

                                            geometry  \
0  POLYGON Z ((-92.97180 37.23656 0.00000, -92.97...   
0  POLYGON Z ((-92.84921 37.07314 0.00000, -92.84...   
0  POLYGON Z ((-92.89152 37.40754 0.00000, -92.89...   
0  POLYGON Z ((-93.07678 37.27042 0.00000, -93.07...   
0  POLYGON Z ((-94.80548 39.82132 0.00000, -94.80...   

                           unique_ID  
0          225East Ozark/Northview A  
0                225Finley/Hazelwood  
0                   225Niangua/Union  
0                225Washington/Grant  
0  003John Glenn/Country Club/Beeler  
                                            geometry  \
0  POLYGON Z ((-92.84921 37.07314 0.00000, -92.84...   
0  POLYGON Z ((-92.89152 37.40754 0.00000, -92.89...   
0  POLYGON Z ((-93.07678 37.27042 0.00000, -93.07...   
0  POLYGON Z ((-94.80548 39.82132 0.00000, -94.80...   
0  POLYGON Z ((-94.89910 40.05581 0.00000, -94.89...   

                           unique_ID  
0                225Finley/Hazelwood  
0             

                                            geometry unique_ID
0  POLYGON Z ((-92.15441 39.20498 0.00000, -92.15...  01910/12
0  MULTIPOLYGON Z (((-92.31240 38.99599 0.00000, ...  01915/40
0  POLYGON Z ((-92.57016 38.97973 0.00000, -92.57...  01918/19
0  POLYGON Z ((-92.35759 38.95292 0.00000, -92.35...  0191C/1G
0  POLYGON Z ((-92.32201 38.94214 0.00000, -92.32...  0191E/6B
                                            geometry    unique_ID
0  MULTIPOLYGON Z (((-92.31240 38.99599 0.00000, ...     01915/40
0  POLYGON Z ((-92.57016 38.97973 0.00000, -92.57...     01918/19
0  POLYGON Z ((-92.35759 38.95292 0.00000, -92.35...     0191C/1G
0  POLYGON Z ((-92.32201 38.94214 0.00000, -92.32...     0191E/6B
0  POLYGON Z ((-92.38073 38.95483 0.00000, -92.38...  0191F/1J/1L
                                            geometry    unique_ID
0  POLYGON Z ((-92.57016 38.97973 0.00000, -92.57...     01918/19
0  POLYGON Z ((-92.35759 38.95292 0.00000, -92.35...     0191C/1G
0  POLYGON Z ((-92.32201 38.

                                            geometry    unique_ID
0  POLYGON Z ((-92.36125 38.92614 0.00000, -92.35...  0194F/4I/5A
0  POLYGON Z ((-92.38293 38.93003 0.00000, -92.38...     0194H/4N
0  POLYGON Z ((-92.47617 38.90446 0.00000, -92.47...  0194K/4M/37
0  POLYGON Z ((-92.34437 38.92429 0.00000, -92.34...     0195C/5H
0  MULTIPOLYGON Z (((-92.36921 38.88680 0.00000, ...  0195D/5G/5M
                                            geometry    unique_ID
0  POLYGON Z ((-92.38293 38.93003 0.00000, -92.38...     0194H/4N
0  POLYGON Z ((-92.47617 38.90446 0.00000, -92.47...  0194K/4M/37
0  POLYGON Z ((-92.34437 38.92429 0.00000, -92.34...     0195C/5H
0  MULTIPOLYGON Z (((-92.36921 38.88680 0.00000, ...  0195D/5G/5M
0  MULTIPOLYGON Z (((-92.40996 38.89974 0.00000, ...     0195E/5P
                                            geometry    unique_ID
0  POLYGON Z ((-92.47617 38.90446 0.00000, -92.47...  0194K/4M/37
0  POLYGON Z ((-92.34437 38.92429 0.00000, -92.34...     0195C/5H
0  MULTIPO

                                            geometry  \
0  MULTIPOLYGON Z (((-90.41087 38.76120 0.00000, ...   
0  POLYGON Z ((-90.37953 38.71171 0.00000, -90.37...   
0  POLYGON Z ((-90.39811 38.73719 0.00000, -90.39...   
0  POLYGON Z ((-90.39037 38.73326 0.00000, -90.39...   
0  POLYGON Z ((-90.33176 38.72225 0.00000, -90.33...   

                                    unique_ID  
0                        189AP019/NW005/NW017  
0                      189AP022/MID007/MID022  
0                              189AP028/AP047  
0                        189AP029/AP031/AP033  
0  189AP003/AP027/NRW002/NRW008/NRW015/NRW029  
                                            geometry  \
0  POLYGON Z ((-90.37953 38.71171 0.00000, -90.37...   
0  POLYGON Z ((-90.39811 38.73719 0.00000, -90.39...   
0  POLYGON Z ((-90.39037 38.73326 0.00000, -90.39...   
0  POLYGON Z ((-90.33176 38.72225 0.00000, -90.33...   
0  MULTIPOLYGON Z (((-90.38222 38.74204 0.00000, ...   

                                    un

                                            geometry                unique_ID
0  POLYGON Z ((-90.46028 38.50187 0.00000, -90.46...         189BON037/BON039
0  POLYGON Z ((-90.40699 38.55549 0.00000, -90.40...  189BON040/GRA002/GRA009
0  POLYGON Z ((-90.41129 38.56702 0.00000, -90.41...  189BON005/BON024/BON036
0  POLYGON Z ((-90.43961 38.58005 0.00000, -90.43...         189BON008/BON022
0  POLYGON Z ((-90.47093 38.66260 0.00000, -90.47...           189CC001/CC010
                                            geometry  \
0  POLYGON Z ((-90.40699 38.55549 0.00000, -90.40...   
0  POLYGON Z ((-90.41129 38.56702 0.00000, -90.41...   
0  POLYGON Z ((-90.43961 38.58005 0.00000, -90.43...   
0  POLYGON Z ((-90.47093 38.66260 0.00000, -90.47...   
0  MULTIPOLYGON Z (((-90.36348 38.66119 0.00000, ...   

                                           unique_ID  
0                            189BON040/GRA002/GRA009  
0                            189BON005/BON024/BON036  
0                             

...

                                            geometry  \
0  POLYGON Z ((-90.34701 38.68561 0.00000, -90.34...   
0  POLYGON Z ((-90.36511 38.71881 0.00000, -90.36...   
0  POLYGON Z ((-90.34333 38.68431 0.00000, -90.34...   
0  MULTIPOLYGON Z (((-90.39422 38.70840 0.00000, ...   
0  POLYGON Z ((-90.37408 38.68288 0.00000, -90.37...   

                                      unique_ID  
0                              189MID002/MID031  
0                              189MID021/MID047  
0  189MID025/MID030/MID032/MID038/NOR028/NOR054  
0                              189MID033/MID061  
0                              189MID036/MID048  
                                            geometry  \
0  POLYGON Z ((-90.36511 38.71881 0.00000, -90.36...   
0  POLYGON Z ((-90.34333 38.68431 0.00000, -90.34...   
0  MULTIPOLYGON Z (((-90.39422 38.70840 0.00000, ...   
0  POLYGON Z ((-90.37408 38.68288 0.00000, -90.37...   
0  POLYGON Z ((-90.37186 38.68842 0.00000, -90.37...   

                          

                                            geometry  \
0  MULTIPOLYGON Z (((-90.45991 38.57583 0.00000, ...   
0  POLYGON Z ((-90.28401 38.69502 0.00000, -90.28...   
0  MULTIPOLYGON Z (((-90.29950 38.70140 0.00000, ...   
0  POLYGON Z ((-90.30158 38.71994 0.00000, -90.30...   
0  POLYGON Z ((-90.31704 38.69290 0.00000, -90.31...   

                                           unique_ID  
0  189MR008/MR012/MR015/MR024/MR033/MR041/MR047/M...  
0                                   189NOR001/NOR002  
0                     189NOR011/NOR039/NOR040/NOR042  
0                     189NOR012/NOR013/NOR017/NOR018  
0                     189NOR014/NOR016/NOR030/NOR050  
                                            geometry  \
0  POLYGON Z ((-90.28401 38.69502 0.00000, -90.28...   
0  MULTIPOLYGON Z (((-90.29950 38.70140 0.00000, ...   
0  POLYGON Z ((-90.30158 38.71994 0.00000, -90.30...   
0  POLYGON Z ((-90.31704 38.69290 0.00000, -90.31...   
0  MULTIPOLYGON Z (((-90.32576 38.70143 0.00000, ... 

                                            geometry  \
0  MULTIPOLYGON Z (((-90.43438 38.73593 0.00000, ...   
0  POLYGON Z ((-90.39497 38.78069 0.00000, -90.39...   
0  MULTIPOLYGON Z (((-90.45273 38.72912 0.00000, ...   
0  POLYGON Z ((-90.37886 38.78414 0.00000, -90.37...   
0  POLYGON Z ((-90.43314 38.73885 0.00000, -90.43...   

                          unique_ID  
0              189NW011/NW020/NW047  
0  189NW018/NW024/NW025/NW030/NW044  
0              189NW019/NW021/NW035  
0                    189NW023/NW034  
0                    189NW026/NW043  
                                            geometry  \
0  POLYGON Z ((-90.39497 38.78069 0.00000, -90.39...   
0  MULTIPOLYGON Z (((-90.45273 38.72912 0.00000, ...   
0  POLYGON Z ((-90.37886 38.78414 0.00000, -90.37...   
0  POLYGON Z ((-90.43314 38.73885 0.00000, -90.43...   
0  POLYGON Z ((-90.40013 38.74404 0.00000, -90.40...   

                          unique_ID  
0  189NW018/NW024/NW025/NW030/NW044  
0              189NW01

                                            geometry  \
0  POLYGON Z ((-90.50046 38.58513 0.00000, -90.50...   
0  POLYGON Z ((-90.52065 38.54699 0.00000, -90.52...   
0  MULTIPOLYGON Z (((-90.51140 38.54646 0.00000, ...   
0  POLYGON Z ((-90.52894 38.58631 0.00000, -90.52...   
0  POLYGON Z ((-90.52960 38.57675 0.00000, -90.52...   

                        unique_ID  
0                189QUE002/QUE003  
0                189QUE021/QUE033  
0  189QUE025/QUE028/QUE034/QUE038  
0                189QUE026/QUE027  
0                 189QUE045/WH041  
                                            geometry  \
0  POLYGON Z ((-90.52065 38.54699 0.00000, -90.52...   
0  MULTIPOLYGON Z (((-90.51140 38.54646 0.00000, ...   
0  POLYGON Z ((-90.52894 38.58631 0.00000, -90.52...   
0  POLYGON Z ((-90.52960 38.57675 0.00000, -90.52...   
0  MULTIPOLYGON Z (((-90.52287 38.54890 0.00000, ...   

                        unique_ID  
0                189QUE021/QUE033  
0  189QUE025/QUE028/QUE034/QUE038  
0 

                                            geometry                unique_ID
0  POLYGON Z ((-90.20853 38.76830 0.00000, -90.20...           189SF007/SF033
0  MULTIPOLYGON Z (((-90.29685 38.81990 0.00000, ...         189SPL010/SPL027
0  MULTIPOLYGON Z (((-90.29082 38.81338 0.00000, ...         189SPL014/SPL024
0  POLYGON Z ((-90.27864 38.79371 0.00000, -90.27...  189SPL015/SPL021/SPL022
0  POLYGON Z ((-90.23023 38.79572 0.00000, -90.23...         189SPL017/SPL023
                                            geometry                unique_ID
0  MULTIPOLYGON Z (((-90.29685 38.81990 0.00000, ...         189SPL010/SPL027
0  MULTIPOLYGON Z (((-90.29082 38.81338 0.00000, ...         189SPL014/SPL024
0  POLYGON Z ((-90.27864 38.79371 0.00000, -90.27...  189SPL015/SPL021/SPL022
0  POLYGON Z ((-90.23023 38.79572 0.00000, -90.23...         189SPL017/SPL023
0  POLYGON Z ((-90.24062 38.79814 0.00000, -90.24...         189SPL002/SPL025
                                            geometry            

In [311]:
#Attempt to join on the "unique_ID" column
census_join = pd.merge(elections_census_shapes,shapefiles_census,how="outer",on="unique_ID",indicator=True)
print(census_join["_merge"].value_counts())
census_join[census_join["_merge"]=="left_only"].to_csv("./census_join/election.csv")
census_join[census_join["_merge"]=="right_only"].to_csv("./census_join/shapes.csv")

both          2089
right_only     340
left_only      171
Name: _merge, dtype: int64


Missouri
--------

Precinct mergers were made in the following counties to match county reporting units: 
- Andrew: Precinct mergers to match county reporting units
- Barry: Precinct mergers to match county reporting units
- Barry: Add Monett City; Align Cassville with corporate boundary
- Barton: Adjust Lamar ward boundaries to match city map
- Benton: Precinct mergers to match county reporting units
- Bollinger: Precinct mergers to match county reporting units
- Bollinger: Adjust Patton/Sedgewickville to match county GIS
- Butler: Align Poplar Bluff with corporate boundary and adjust wards to match city map
- Boone: Precinct mergers to match county reporting units
- Carroll: Precinct mergers to match county reporting units
- Carroll: Adjust Carrollton City wards to match voter file
- Carter: Precinct mergers to match county reporting units
- Cass: Precinct mergers to match county reporting units
- Cass: Align Belton, Pleasant Hill, Raymore precincts with corporate boundaries; Adjust 37 Pleasant Prairie/25 West Peculiar Rural
- Chariton: Adjust Salisbury wards to match city map
- Christian: Revise Cassidy 1/Cassidy 2; Reverse Nixa City annexation into Rosedale 1B
- Christian: Precinct mergers to match county reporting units
- Clark: Precinct mergers to match county reporting units
- Clay: Adjust Excelsior Springs, Kearney, Smithville precincts to match 2018 ward and corporate boundaries; Adjust KC 21-1/15, 21-1/17, 21-3/11, 21-4/6, 21-4/11, 21-5/7/24, 21-6/24, 21-7/20, 21-13/26, 21-14/23, 21-17/16, 21-17/26, 21-17/Platte 1, Gallatin 7/16, 10/13, 10/16, 11/15, 13/15, Kearney 3/Platte 2, Liberty 1/10, 6/13, 7/11, 11/14, Washington 1/3 to match voter file
- Clinton: Align Atchison/Concord, Jackson/Lathrop with 2018 precinct split assignments
- Cole: Precinct mergers to match county reporting units
- Cole: Adjust Marion Twp St Martins to include St Martins city in Jefferson Twp
- Crawford: Move Benton Twp section of Leasburg precinct from Recklein to Leasburg
- Crawford: Precinct mergers to match county reporting units
- Dade: Realign Greenfield wards to match municipal code
- Dallas: Split N Benton Inside/OCL, S Benton Inside/OCL; Align N/S Benton with Buffalo ward map
- Dallas: Precinct mergers to match county reporting units
- Dekalb: Precinct mergers to match county reporting units
- Dent: Split Boss precinct from R-4 precinct; Adjust R-2/R-3 boundary
- Douglas: Precinct mergers to match county reporting units
- Dunklin: Precinct mergers to match county reporting units
- Dunklin: Align Holcomb, Rives with township boundaries; Adjust Arbyrd/Senath; Align Kennett wards with city map
- Gasconade: Align Hermann, Owensville, Rosebud with corporate boundaries and ward descriptions
- Gentry: Precinct mergers to match county reporting units
- Grundy: Precinct mergers to match county reporting units
- Holt: Precinct mergers to match county reporting units
- Holt: Split West between Central/Northwest by school district
- Howard: Precinct mergers to match county reporting units
- Howell: Precinct mergers to match county reporting units
- Howell: Align West Plains precincts with 2018 corporate boundary
- Iron: Adjust Annapolis/Des Arc/Vulcan, Ironton/Pilot Knob
- Johnson: Adjust Warrensburg NE/NW boundary to match county maps
- Johnson: Precinct mergers to match county reporting units
- Lewis: Split LaBelle Twp between LaBelle FS/Lewistown FS with PLSS grid
- Lincoln: Align Moscow Mills, Troy with 2018 corporate boundaries; Adjust Briscoe/Troy Rural
- Linn: Precinct mergers to match county reporting units
- Linn: Split Brookfield Township between Brookfield 1&2/3&4
- Livingston: Align Chillicothe with 2018 corporate boundaries
- Livingston: Precinct mergers to match county reporting units
- Macon: Align Macon City wards with corporate boundaries
- Madison: Align Fredericktown wards with corporate boundaries
- Mercer: Precinct mergers to match county reporting units
- Mississippi: Precinct mergers to match county reporting units
- Mississippi: Adjust Bertrand/East Prairie Rural; Align East Prairie City with corporate boundary
- Moniteau: Precinct mergers to match county reporting units
- Montgomery: Align Bellflower/Middletown, Big Spring/Rhineland with voter file
- Morgan: Adjust Versaille North/South boundary to match county GIS
- Morgan: Precinct mergers to match county reporting units
- New Madrid: Precinct mergers to match county reporting units
- New Madrid: Add New Madrid City wards; Add Big Prairie 4 precinct; Move Portageville Ward 1 to Portage 1
- Newton: Align Staples precincts with Joplin City GIS boundaries; Align Neosho precincts with Neosho City GIS boundaries; Adjust Newtonia/Ritchey Wentworth
- Oregon: Precinct mergers to match county reporting units
- Pettis: Precinct mergers to match county reporting units
- Pettis: Adjust Sedalia City wards to match city map
- Phelps: Align St. James ward boundaries with city map
- Phelps: Precinct mergers to match county reporting units
- Polk: Precinct mergers to match county reporting units
- Polk: Split Campbell/Jefferson precincts by school district
- Putnam: Split Union Twp into NE/SW precincts to match county map
- Putnam: Precinct mergers to match county reporting units
- Pulaski: Adjust Big Piney/St. Roberts, Laquey/Waynesville
- Ralls: Precinct mergers to match county reporting units
- Ralls: Replace Center/Liberty precincts with 2010 VTDs
- Ray: Adjust Richmond City wards, Homestead Village/Wood Heights/Lawson, Richmond/Orrick
- Ray: Precinct mergers to match county reporting units
- Reynolds: Realign precincts countywide to match voter file
- Ripley: Precinct mergers to match county reporting units
- Saline: Split Nelson precinct from Hardeman precinct; Adjust Marshall Ward 2/3 boundary
- St. Charles: Adjust Brian/Ridgepoint, Civic/St. Paul, Harvester/Sycamore, Mamelle/Marina
- St. Clair: Precinct mergers to match county reporting units
- St. Francois: Precinct mergers to match county reporting units
- St. Francois: Align Desloge, Farmington, Park Hills with corporate boundaries and city ward maps; Adjust Bonne Terre/Timberline, Patterson/Timberline, Leadwood/Park Hills to match voter file
- St. Louis: Precinct mergers to match county reporting units
- St. Louis: Add UNV044 precinct; Split NRW011/NRW052
- Schuyler: Precinct mergers to match county reporting units
- Shannon: Precinct mergers to match county reporting units
- Shannon: Align all precincts with county GIS and voter file
- Shelby: Precinct mergers to match county reporting units
- Shelby: Adjust Jackson/North River to match voter file
- Stoddard: Precinct mergers to match county reporting units
- Stoddard: Align Advance, Bernie, Bloomfield, Dexter, Puxico with corporate boundaries; Adjust Advance/Leora, Bernie/Dexter/Pyle, Bloomfield/Gray Ridge
- Stone: Move Branson West from Ruth A to Ruth C; Split Ruth C Rural/Sunset Cove; Adjust Ruth B City/Rural to match county GIS
- Sullivan: Precinct mergers to match county reporting units
- Taney: Adjust Boston Center/Walnut Shade; Align Branson wards with 2018 corporate boundary
- Vernon: Align Bronaugh/Moundville, Harwood/Schell City, Deerfield/NG Armory with voter file
- Wayne: Precinct mergers to match county reporting units
- Webster: Precinct mergers to match county reporting units





### 2. 2010 Census VTD release
>Monroe County comes from the 2010 Census VTD release.

https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2010&layergroup=Voting+Districts

In [ ]:
monroe_shape = gp.read_file("./raw-from-source/Census/2010_VTD/tl_2010_29137_vtd10/tl_2010_29137_vtd10.shp")

### 3. 2020 Census VTD release
>Platte County comes from the 2020 Census VTD release.

https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html

In [ ]:
platte_shape = gp.read_file("./raw-from-source/Census/2020_VTD/tl_2020_29_all/tl_2020_29165_vtd20.shp")
platte_shape.plot()

- Platte: Precinct mergers to match county reporting units
- Platte: Align Lake Waukomis, Parkville, Platte City, Riverside/Northmore, Weatherby Lake with corporate boundaries; Adjust Platte Hills/Park Hill, Seven Bridges/Shiloh

### 4. Counties themselves
> The following counties instead used maps or shapefiles sourced from the respective county governments:  
Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.  

Able to find shapefiles for Camden, Cooper, Marion, Lafayette, Laclede, Jasper, Bates, Audrain.  

Of these, only Jasper came in shapefile format. The others came in tif formats that I was unable to load into this notebook without digitizing.

In [ ]:
jasper_precincts = gp.read_file("./raw-from-source/Counties/Jasper/Voting_Precincts_2020-shp/Voting_Precincts_2020.shp")
print(jasper_precincts.shape)

In [ ]:
jasper_precincts.plot()

- Audrain: https://audraingis.integritygis.com/H5/Index.html?viewer=audrain
- Bates: https://batesgis.integritygis.com/H5/Index.html?viewer=bates
- Caldwell: can't find
- Callaway: can't find
- Camden: https://camdengis.integritygis.com/H5/Index.html?viewer=camden
- Cape Girardeau: only pdf (https://capecountyelections.com/wp-content/uploads/2019/06/Voter_Precincts.pdf)
- Cedar: can't find
- Cooper: https://coopergis.integritygis.com/H5/Index.html?viewer=cooper
- Franklin: can't find
- Greene: not available (https://www.greenecountyassessor.org/assessor/maps/mapadv.aspx)
- Jackson: not downloadable (https://jcebmo.org/wp-content/uploads/JACKSON-COUNTY-TOWNSHIPS-PRECINCTS.pdf)
- Jasper: https://jaspercountymogisintiatives-jcmo.hub.arcgis.com/datasets/voting-precincts-2020
- Jefferson: can't find
- Laclede: https://lacledegis.integritygis.com/H5/Index.html?viewer=laclede
- Lafayette: https://lafayettegis.integritygis.com/H5/Index.html?viewer=lafayette
- Lawrence: not downloadable (https://www.lawrencecountymo.org/elections)
- Marion: https://mariongis.integritygis.com/H5/Index.html?viewer=marion_public
- McDonald: can't find
- Miller: not available (https://millergis.integritygis.com/H5/Index.html?viewer=miller)
- Nodaway: can't find
- Osage: can't find
- Ozark: can't find
- Pemiscot: can't find
- Pike: can't find
- Randolph: can't find
- Scott: not downloadable https://www.arcgis.com/apps/webappviewer/index.html?id=73de0cc0034040a5b8b0f5988a643164
- Ste. Genevieve: can't find
- Texas: can't find
- Warren: not downloadable (https://www.arcgis.com/apps/webappviewer/index.html?id=edfaf72f348b4ea28a959815d82dfe86)
- Washington: can't find
- Worth: can't find
- Wright: can't find

### Shapefile Modifications

- Audrain: Precinct mergers to match county reporting units
- Audrain: Add Vandalia City; Align Mexico precincts with municipal code
  
- Callaway: Align Fulton, Holts Summit precincts with 2018 corporate boundaries
- Callaway: Precinct mergers to match county reporting units

- Camden: Precinct mergers to match county reporting units

- Cape Girardeau: Precinct mergers to match county reporting units

- Cooper: Precinct mergers to match county reporting units

- Franklin: Split Dry Branch between Stanton & St. Clair Out of Town
- Franklin: Precinct mergers to match county reporting units

- Greene: Precinct mergers to match county reporting units

- Jackson: Precinct mergers to match county reporting units

- Jasper: Precinct mergers to match county reporting units
- Jasper: Realign Carthage 2/3, Joplin 1/7, 10/13, 10/15, 13/22, Oronogo 1/2, Webb City 1/4 to match city ward maps

- Jefferson: Precinct mergers to match county reporting units
- Jefferson: Split Festus Ward 1 P1/P2, Ward 3 P1/P2, Ward 4 P1/P2; Adjust Crystal City/Riverview

- Marion: Precinct mergers to match county reporting units
- Marion: Split Fabius-Smileyville/Taylor, Warren-Monroe City/Warren; Align Hannibal wards with voter file

- Miller: Adjust all precinct boundaries to match voter file

- Nodaway: Precinct mergers to match county reporting units

- Osage: Precinct mergers to match county reporting units

- Pemiscot: Precinct mergers to match county reporting units

- Pike: Adjust Cuivre JL boundaries with Ashley NO, Buffalo E, Prairieville I

- Randolph: Precinct mergers to match county reporting units

- Scott: Adjust Haywood City/Morley, Sikeston Ward 1/McMullin/Vanduser

- Warren: Precinct mergers to match county reporting units

- Washington: Adjust City Hall/Stony Point, Cruise/Richwoods to match county GIS

- Worth: Precinct mergers to match county reporting units